In [ ]:
# Single run
from floor_generator import *

hyperparams = {
    'east_joists' : MemberSpec('c24_80x160', quantity=1, padding=190),
    'west_joists' : MemberSpec('c24_60x120', quantity=1, padding=130),
    'tail_joists' : MemberSpec('c24_80x160', quantity=1, padding=0),
    'trimmerW' : MemberSpec('c24_80x160', quantity=1),
    'trimmerE' : MemberSpec('c24_80x160', quantity=1),
    'header' : MemberSpec('c24_60x120', quantity=1),
    'planks' : MemberSpec('c18_200x25'),
    }

frame, nodes, members = create_model(hyperparams, walls=True, exclude_loads_under_big_beam=True, free_tail=True)
member_evaluations = evaluate_stresses(frame, members)
total_cost, cuts = calculate_purchase_quantity(frame, members)
# member_evaluations = group_stresses_by_member(part_evaluations, join_planks=False)

# Optimize to target ratio
target = 0
above_target_multiplier = 2.0
below_target_multiplier = 1.0

diff = member_evaluations - target
multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
member_evaluations_penalty = multiplier * (diff * diff)

max_ratio = member_evaluations_penalty.max().max()
mean_ratio = member_evaluations_penalty.mean().mean()
score = total_cost * mean_ratio * max_ratio

nodes

In [ ]:
# Single run
from floor_generator import *

hyperparams = {
    'east_joists' : MemberSpec('c24_80x160', quantity=1, padding=190),
    'west_joists' : MemberSpec('c24_60x120', quantity=1, padding=140),
    'tail_joists' : MemberSpec('c24_80x160', quantity=1, padding=0),
    'trimmerW' : MemberSpec('c24_80x160', quantity=1),
    'trimmerE' : MemberSpec('c24_80x160', quantity=1),
    'header' : MemberSpec('c24_60x120', quantity=1),
    'planks' : MemberSpec('c18_200x25'),
    }

frame, nodes, members = create_model(hyperparams, walls=True, exclude_loads_under_big_beam=True, free_tail=True)
member_evaluations = evaluate_stresses(frame, members)
total_cost, cuts = calculate_purchase_quantity(frame, members)
# member_evaluations = group_stresses_by_member(part_evaluations, join_planks=False)

# Optimize to target ratio
target = 0
above_target_multiplier = 2.0
below_target_multiplier = 1.0

diff = member_evaluations - target
multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
member_evaluations_penalty = multiplier * (diff * diff)

max_ratio = member_evaluations_penalty.max().max()
mean_ratio = member_evaluations_penalty.mean().mean()
score = total_cost * mean_ratio * max_ratio

print(score)
member_evaluations

In [ ]:
# Full optimization - No beam detected
import nevergrad as ng
import pandas as pd
import numpy as np
import warnings
import math
from tqdm import tqdm
from floor_generator import *


def objective_function(east_material, east_quantity, east_padding,
                      west_material, west_quantity, west_padding,
                      tail_material, tail_quantity, tail_padding,
                      trimmerW_material, trimmerE_material, header_material, plank_material):
    
    param_dict = {
        'east_material': east_material,
        'east_quantity': east_quantity,
        'east_padding': east_padding,
        'west_material': west_material,
        'west_quantity': west_quantity,
        'west_padding': west_padding,
        'tail_material': tail_material,
        'tail_quantity': tail_quantity,
        'tail_padding': tail_padding,
        'trimmerW_material': trimmerW_material,
        'trimmerE_material': trimmerE_material,
        'header_material': header_material,
        'plank_material': plank_material,
    }
    
    hyperparams = {
        'east_joists': MemberSpec(east_material, quantity=east_quantity, padding=east_padding),
        'west_joists': MemberSpec(west_material, quantity=west_quantity, padding=west_padding),
        'tail_joists': MemberSpec(tail_material, quantity=tail_quantity, padding=tail_padding),
        'trimmerW': MemberSpec(trimmerW_material, quantity=1),
        'trimmerE': MemberSpec(trimmerE_material, quantity=1),
        'header': MemberSpec(header_material, quantity=1),
        'planks': MemberSpec(plank_material),
    }

    frame, _, members = create_model(hyperparams, walls=True, exclude_loads_under_big_beam=True)
    part_evaluations = evaluate_stresses(frame, members)
    total_cost, cuts = calculate_purchase_quantity(frame, members)
    member_evaluations = group_stresses_by_member(part_evaluations, join_planks=True)
    member_evaluations = member_evaluations.drop('net_deflection', axis=1)

    # Optimize to target ratio
    target = 0.9
    above_target_multiplier = 2.0
    below_target_multiplier = 1.0

    diff = member_evaluations - target
    multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
    member_evaluations_penalty = multiplier * (diff * diff)

    variety_factor = math.log(len(cuts), 99) + 1

    max_ratio = member_evaluations_penalty.max().max()
    mean_ratio = member_evaluations_penalty.mean().mean()
    score = total_cost * mean_ratio * max_ratio * variety_factor


    run_result = {
        **param_dict,
        'total_cost': total_cost,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'score': score,
        'cuts': cuts,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS.append(run_result)

    return score


def optimize_with_nevergrad(space_config, n_calls=2048, n_workers=1, algorithm='NGOpt'):
    param_dict = {}
    for param_name, param_spec in space_config.items():
        if param_spec['type'] == 'categorical':
            param_dict[param_name] = ng.p.Choice(param_spec['choices'])
        elif param_spec['type'] == 'integer':
            param_dict[param_name] = ng.p.Scalar(lower=param_spec['lower'], upper=param_spec['upper']).set_integer_casting()

    parametrization = ng.p.Instrumentation(**param_dict)
    
    # Select optimizer
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=n_workers)
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function(**x.kwargs)
        optimizer.tell(x, value)
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


# Units are mm, N, and MPa (N/mm²)
# INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

west_space = INPUT_PARAMS.opening_x_start
east_space = INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)
tail_space = INPUT_PARAMS.opening_length

max_west_quantity, max_east_quantity, max_tail_quantity = 3, 3, 4

beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
max_west_padding = west_space - (beam_max_base * max_west_quantity)
max_east_padding = east_space - (beam_max_base * max_east_quantity)
max_tail_padding = (tail_space - (beam_max_base * max_tail_quantity))/2

space_config = {
    'east_material': {'type': 'categorical', 'choices': beam_ids},
    'east_quantity': {'type': 'integer', 'lower': 1, 'upper': max_east_quantity},
    'east_padding': {'type': 'integer', 'lower': 0, 'upper': max_east_padding},
    
    'west_material': {'type': 'categorical', 'choices': beam_ids},
    'west_quantity': {'type': 'integer', 'lower': 1, 'upper': max_west_quantity},
    'west_padding': {'type': 'integer', 'lower': 0, 'upper': max_west_padding},
    
    'tail_material': {'type': 'categorical', 'choices': beam_ids},
    'tail_quantity': {'type': 'integer', 'lower': 1, 'upper': max_tail_quantity},
    'tail_padding': {'type': 'integer', 'lower': 0, 'upper': max_tail_padding},
    
    'trimmerW_material': {'type': 'categorical', 'choices': beam_ids},
    'trimmerE_material': {'type': 'categorical', 'choices': beam_ids},
    'header_material': {'type': 'categorical', 'choices': beam_ids},
    'plank_material': {'type': 'categorical', 'choices': floor_ids},
}

n_calls = 4000
_GLOBAL_EVALUATIONS = []

result = optimize_with_nevergrad(
    space_config, 
    n_calls=n_calls,
    n_workers=8,
    algorithm='NGOpt'  # Try also: 'CMA', 'NGOpt', 'TwoPointsDE', 'PSO', 'DE'
)

results_df = pd.DataFrame(_GLOBAL_EVALUATIONS)
results_df = results_df.sort_values(by='score')
results_df

In [ ]:
# Full optimization - beam considered
import nevergrad as ng
import pandas as pd
import numpy as np
import warnings
import math
from tqdm import tqdm
from floor_generator import *


def objective_function(east_material, east_padding,# east_quantity,
                      west_material, west_padding,# west_quantity,
                      tail_material, tail_padding,# tail_quantity,
                      trimmerW_material, trimmerE_material, header_material):
    
    param_dict = {
        'east_material': east_material,
        'east_padding': east_padding,
        # 'east_quantity': east_quantity,
        'west_material': west_material,
        'west_padding': west_padding,
        # 'west_quantity': west_quantity,
        'tail_material': tail_material,
        'tail_padding': tail_padding,
        # 'tail_quantity': tail_quantity,
        'trimmerW_material': trimmerW_material,
        'trimmerE_material': trimmerE_material,
        'header_material': header_material,
    }
    
    hyperparams = {
        'east_joists': MemberSpec(east_material, quantity=1, padding=east_padding),
        'west_joists': MemberSpec(west_material, quantity=1, padding=west_padding),
        'tail_joists': MemberSpec(tail_material, quantity=1, padding=tail_padding),
        'trimmerW': MemberSpec(trimmerW_material, quantity=1),
        'trimmerE': MemberSpec(trimmerE_material, quantity=1),
        'header': MemberSpec(header_material, quantity=1),
        'planks': MemberSpec('c18_200x25'),
    }

    frame, _, members = create_model(hyperparams, walls=True, exclude_loads_under_big_beam=True)#, free_tail=True)
    member_evaluations = evaluate_stresses(frame, members)
    total_cost, cuts = calculate_purchase_quantity(frame, members)
    # member_evaluations = group_stresses_by_member(part_evaluations, join_planks=False)
    member_evaluations = member_evaluations.drop('net_deflection', axis=1)

    # Optimize to target ratio
    target = 1
    above_target_multiplier = 2.0
    below_target_multiplier = 1.0

    diff = member_evaluations - target
    multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
    member_evaluations_penalty = multiplier * (diff * diff)

    variety_factor = math.log(len(cuts), 99) + 1

    max_ratio = member_evaluations_penalty.max().max()
    mean_ratio = member_evaluations_penalty.mean().mean()
    score = total_cost * mean_ratio * max_ratio * variety_factor

    run_result = {
        **param_dict,
        'total_cost': total_cost,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'variety_factor': variety_factor,
        'score': score,
        'cuts': cuts,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS.append(run_result)

    return score


def optimize_with_nevergrad(space_config, n_calls=2048, n_workers=1, algorithm='NGOpt'):
    param_dict = {}
    for param_name, param_spec in space_config.items():
        if param_spec['type'] == 'categorical':
            param_dict[param_name] = ng.p.Choice(param_spec['choices'])
        elif param_spec['type'] == 'integer':
            param_dict[param_name] = ng.p.Scalar(lower=param_spec['lower'], upper=param_spec['upper']).set_integer_casting()

    parametrization = ng.p.Instrumentation(**param_dict)
    
    # Select optimizer
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=n_workers)
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function(**x.kwargs)
        optimizer.tell(x, value)
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


# Units are mm, N, and MPa (N/mm²)
# INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

west_space = INPUT_PARAMS.opening_x_start
east_space = INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)
tail_space = INPUT_PARAMS.opening_length

max_west_quantity, max_east_quantity, max_tail_quantity = 1,1,1

beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
max_west_padding = west_space - (beam_max_base * max_west_quantity)
max_east_padding = east_space - (beam_max_base * max_east_quantity)
max_tail_padding = (tail_space - (beam_max_base * max_tail_quantity))/2

space_config = {
    'east_material': {'type': 'categorical', 'choices': beam_ids},
    # 'east_quantity': {'type': 'integer', 'lower': 1, 'upper': max_east_quantity},
    'east_padding': {'type': 'integer', 'lower': 1, 'upper': max_east_padding},
    
    'west_material': {'type': 'categorical', 'choices': beam_ids},
    # 'west_quantity': {'type': 'integer', 'lower': 1, 'upper': max_west_quantity},
    'west_padding': {'type': 'integer', 'lower': 1, 'upper': max_west_padding},
    
    'tail_material': {'type': 'categorical', 'choices': beam_ids},
    # 'tail_quantity': {'type': 'integer', 'lower': 1, 'upper': max_tail_quantity},
    'tail_padding': {'type': 'integer', 'lower': 1, 'upper': max_tail_padding},
    
    'trimmerW_material': {'type': 'categorical', 'choices': beam_ids},
    'trimmerE_material': {'type': 'categorical', 'choices': beam_ids},
    'header_material': {'type': 'categorical', 'choices': beam_ids},
}

n_calls = 5000
_GLOBAL_EVALUATIONS = []

result = optimize_with_nevergrad(
    space_config, 
    n_calls=n_calls,
    n_workers=8,
    algorithm='NGOpt'  # Try also: 'CMA', 'NGOpt', 'TwoPointsDE', 'PSO', 'DE'
)

results_df = pd.DataFrame(_GLOBAL_EVALUATIONS)
results_df = results_df.sort_values(by='score')
results_df

# Padding refinement

In [ ]:
# Parts
import nevergrad as ng
import pandas as pd
import numpy as np
from tqdm import tqdm
from floor_generator import *


def objective_function(east_padding,
                      west_padding):
    param_dict = {
        'east_padding': east_padding,
        'west_padding': west_padding,
    }
    hyperparams = {
        'east_joists': MemberSpec('c24_80x160', quantity=1, padding=east_padding),
        'west_joists': MemberSpec('c24_60x120', quantity=1, padding=west_padding),
        'tail_joists': MemberSpec('c24_80x160', quantity=1, padding=0),
        'trimmerW': MemberSpec('c24_80x160', quantity=1),
        'trimmerE': MemberSpec('c24_80x160', quantity=1),
        'header': MemberSpec('c24_60x120', quantity=1),
        'planks': MemberSpec('c18_200x25'),
    }

    frame, _, members = create_model(hyperparams, walls=True, exclude_loads_under_big_beam=True)
    member_evaluations = evaluate_stresses(frame, members)
    total_cost, cuts = calculate_purchase_quantity(frame, members)
    # member_evaluations = group_stresses_by_member(part_evaluations, join_planks=False)
    member_evaluations = member_evaluations.drop('net_deflection', axis=1)

    # Optimize to target ratio
    target = 0
    above_target_multiplier = 2.0
    below_target_multiplier = 1.0

    diff = member_evaluations - target
    multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
    member_evaluations_penalty = multiplier * (diff * diff)

    max_ratio = member_evaluations_penalty.max().max()
    mean_ratio = member_evaluations_penalty.mean().mean()
    score = total_cost * mean_ratio * max_ratio

    run_result = {
        **param_dict,
        'total_cost': total_cost,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'score': score,
        'cuts': cuts,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS.append(run_result)

    return score


def optimize_with_nevergrad(space_config, n_calls=2048, n_workers=1, algorithm='NGOpt'):
    param_dict = {}
    for param_name, param_spec in space_config.items():
        if param_spec['type'] == 'categorical':
            param_dict[param_name] = ng.p.Choice(param_spec['choices'])
        elif param_spec['type'] == 'integer':
            param_dict[param_name] = ng.p.Scalar(lower=param_spec['lower'], upper=param_spec['upper']).set_integer_casting()

    parametrization = ng.p.Instrumentation(**param_dict)
    
    # Select optimizer
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=n_workers)
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function(**x.kwargs)
        optimizer.tell(x, value)
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

west_space = INPUT_PARAMS.opening_x_start
east_space = INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)
tail_space = INPUT_PARAMS.opening_length

max_west_quantity, max_east_quantity, max_tail_quantity = 1, 1, 1

beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
max_west_padding = west_space - (beam_max_base * max_west_quantity)
max_east_padding = east_space - (beam_max_base * max_east_quantity)
max_tail_padding = (tail_space - (beam_max_base * max_tail_quantity))

space_config = {
    'east_padding': {'type': 'integer', 'lower': 50, 'upper': 300},
    'west_padding': {'type': 'integer', 'lower': 50, 'upper': 300},
}

n_calls = 500
_GLOBAL_EVALUATIONS = []

result = optimize_with_nevergrad(
    space_config, 
    n_calls=n_calls,
    n_workers=8,
    algorithm='NGOpt'  # Try also: 'CMA', 'NGOpt', 'TwoPointsDE', 'PSO', 'DE'
)

results_df = pd.DataFrame(_GLOBAL_EVALUATIONS)
results_df = results_df.sort_values(by='score')
results_df